# Construcción de knowledge Base

In [1]:
import json

In [2]:
import requests

url = 'https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py'
response = requests.get(url)

with open('minsearch.py', 'wb') as file:
    file.write(response.content)

In [3]:
# Motor de busqueda creado previamente
import minsearch

In [4]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:

documents= []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
len(documents)
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]
)

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
#Analiza los documentos y obtiene la información que pide index
index.fit(documents)

In [10]:
#Prioriza las respuestas de acuerdo a las preguntas dadas y lo que escanea en los documentos
# Los parametros que se pasan para question y section, es la relevancia (o peso) que se le da a encontrar las palabras en cada una de las secciones señaladas 
boost = {'question':3.0,'section':0.5}

results = index.search(
    query = q,
    filter_dict={'course':'data-engineering-zoomcamp'}, #Filtro de la base de documentos
    boost_dict=boost,
    num_results=5
)

In [11]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# Conectar Chatgpt 4o

In [14]:
from openai import OpenAI
import os

In [15]:
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [19]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user','content':q}]
)

In [21]:
response.choices[0].message.content

"The ability to enroll in a course after it has already started depends on several factors, including the policies of the institution or organization offering the course, the type of course, and how much time has passed since the course began. Here are some steps you can take to find out if you can still enroll:\n\n1. **Check the Course Website:** Many educational institutions provide detailed information about enrollment deadlines and late registration policies on their official websites. \n\n2. **Contact the Instructor:** If the course is managed by a specific instructor or professor, reaching out to them directly can clarify whether late enrollment is possible.\n\n3. **Speak with the Admissions Office:** The admissions or registrar's office can provide information about institutional policies regarding late enrollment and may offer guidance on the steps you need to take.\n\n4. **Review the Syllabus:** If you can access the syllabus or course outline, review it to understand what you

In [32]:
prompt_template ="""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ Database. Use only the facts from the CONTEXT when answering the QUESTION.
if the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
"""

In [35]:
context = ""

for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n".strip()

In [36]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with th

In [38]:
prompt = prompt_template.format(question=q, context=context).strip()

In [39]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ Database. Use only the facts from the CONTEXT when answering the QUESTION.
if the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final

In [40]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role':'user','content':prompt}]
)

response.choices[0].message.content

"Yes, even if the course has already started, you can still enroll and are eligible to submit the homeworks. Be aware that there will be deadlines for turning in the final projects, so don't leave everything for the last minute."